In [4]:
# Linear algebra
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
### plotting
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import cvxpy as cp
import time
### Other
import itertools
from timeit import default_timer as timer
from itertools import product
from ipywidgets import interact, widgets
import warnings
warnings.filterwarnings("ignore")

In [10]:
def LagQUAD_fourier_VG_call_on_min_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R):
  """Compute rainbow options under the VG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.laguerre.laggauss(N)  # Laguerre quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = list(product(*u_1d))  # Full Tensor Isotropic Grid of abcissas
  w_ft = list(product(*w_1d))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(len(u_ft)):  # For all grid points
    u_list = laguerre_combinations(u_ft[i])  # Get combinations of Laguerre terms
    y_list = u_list + 1j * R
    phi_list = np.zeros(len(u_list), dtype=complex)  # List of characteristic function evaluations
    p_list = np.zeros(len(u_list), dtype=complex)  # List of payoff function evaluations
    g_list = np.zeros(len(u_list), dtype=complex)  # List of integrands resulting from change of variables
    for k in range(len(u_list)):  # For all possible combinations within 1 grid point
      phi_list[k] = VG_characteristic_function(y_list[k], SIGMA, T, r, theta, nu)
      p_list[k] = fourier_payoff_call_on_min(y_list[k])
      g_list[k] = np.exp(np.multiply(1j, np.dot(u_list[k], X0))) * phi_list[k] * p_list[k]
    g = np.sum(g_list)
    V = V + discount * np.exp(np.sum(u_ft[i])) * np.prod(w_ft[i]) * g
  return np.real(V)  # Real Part of the integral

def LagQUAD_fourier_VG_basket_put_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R):
  """Compute rainbow options under the VG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.laguerre.laggauss(N)  # Laguerre quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = list(product(*u_1d))  # Full Tensor Isotropic Grid of abcissas
  w_ft = list(product(*w_1d))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, d*K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(len(u_ft)):  # For all grid points
    u_list = laguerre_combinations(u_ft[i])  # Get combinations of Laguerre terms
    y_list = u_list + 1j * R
    phi_list = np.zeros(len(u_list), dtype=complex)  # array of characteristic function evaluations
    p_list = np.zeros(len(u_list), dtype=complex)  # array of payoff function evaluations
    g_list = np.zeros(len(u_list), dtype=complex)  # array of integrands resulting from the change of variables
    for k in range(len(u_list)):  # For all possible combinations within 1 grid point
      phi_list[k] = VG_characteristic_function(y_list[k], SIGMA, T, r, theta, nu)
      p_list[k] = fourier_payoff_basket_put(y_list[k])
      g_list[k] = np.exp(np.multiply(1j, np.dot(u_list[k], X0))) * phi_list[k] * p_list[k]
    g = np.sum(g_list)
    V = V + discount * np.exp(np.sum(u_ft[i])) * np.prod(w_ft[i]) * g
  return np.real(V)  # Real Part of the integral

def HermQUAD_fourier_VG_call_on_min_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R):
  """Compute rainbow options under the VG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.hermite.hermgauss(N)  # Hermite quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = np.asarray(list(product(*u_1d)))  # Full Tensor Isotropic Grid of abcissas
  w_ft = np.asarray(list(product(*w_1d)))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount_factor = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0)  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(u_ft.shape[0]):  # For all grid points
    y = u_ft[i] + 1j * R
    phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)
    payoff = fourier_payoff_call_on_min(y)
    V += np.exp(np.sum(u_ft[i]**2)) * np.prod(w_ft[i]) * np.exp(1j * u_ft[i] @ X0) * phi * payoff
  return np.real(discount_factor * V)  # Real Part of the integral

def HermQUAD_fourier_VG_basket_put_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R):
  """Compute rainbow options under the VG model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - theta (array): Array of skewness parameters of each stock.
  - nu (float): controls the kurtosis of the distribution of log-returns
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.hermite.hermgauss(N)  # Hermite quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = np.asarray(list(product(*u_1d)))  # Full Tensor Isotropic Grid of abcissas
  w_ft = np.asarray(list(product(*w_1d)))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, d*K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount_factor = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0)  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(u_ft.shape[0]):  # For all grid points
    y = u_ft[i] + 1j * R
    phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)
    payoff = fourier_payoff_basket_put(y)
    V += np.exp(np.sum(u_ft[i]**2)) * np.prod(w_ft[i]) * np.exp(1j * u_ft[i] @ X0) * phi * payoff
  return np.real(discount_factor * V)  # Real Part of the integral

def covariance_matrix(sigma, rho):
    """Compute the covariance matrix.
    Args:
    - sigma (array): Array of volatilities of each stock.
    - rho (array): Correlation matrix.
    Returns:
    - SIGMA (array): Covariance matrix.
    """
    sigma = np.diag(sigma)  # Diagonal matrix of volatilities
    SIGMA = np.dot(sigma, np.dot(rho, sigma))  # Covariance matrix calculation
    return SIGMA

def VG_characteristic_function(u, SIGMA, T, r, theta, nu):
    """Calculate the characteristic function of Variance-Gamma process.
    Args:
    - u (array): Vector in Rd.
    - SIGMA (array): Covariance matrix.
    - T (float): Terminal time.
    - r (float): Short rate.
    - theta (array): Array of theta values.
    - nu (float): Nu parameter.
    Returns:
    - phi (complex): Characteristic function value.
    """
    d = len(theta)  # Number of stocks
    w = (1/nu) * np.log(1 - nu * theta - 0.5 * nu * np.diag(SIGMA))  # Martingale correction term
    phi = np.exp(np.multiply(1j * T, np.dot(r + w, u))) * (1 - np.multiply(1j * nu, np.dot(theta, u)) +
                                                           0.5 * nu * np.dot(u, np.dot(SIGMA, u))) ** (-T/nu)
    return phi

def fourier_payoff_call_on_min(u):
    """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
    Args:
    - u (array): Array of Fourier frequencies.
    Returns:
    - payoff (float): Call on min option payoff Fourier transofrm value.
    """
    denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
    return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)


def integrand_to_optimize_VG_call_on_min(R):
    """Calculate the integrand for QMC estimation of the rainbow option under VG model.
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): Integrand value.
    """
    d = len(S0)  # Dimensionality
    X0 = np.log(np.divide(S0, K))  # Element-wise division
    y = np.multiply(1j, R)
    phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Characteristic function
    p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
    integrand = K * discount * phi * p
    return integrand

def integrand_to_optimize_VG_basket_put(R):
    """Calculate the integrand for QMC estimation of the basket put option under VG model.
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): Integrand value.
    """
    d = len(S0)  # Dimensionality
    X0 = np.log(np.divide(S0, d*K))  # Element-wise division
    y = np.multiply(1j, R)
    phi = VG_characteristic_function(y, SIGMA, T, r, theta, nu)  # Characteristic function
    p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
    discount = ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-np.dot(R, X0))  # Modified discount factor
    integrand = K * discount * phi * p
    return integrand

def laguerre_combinations(u):
  """Generate combinations of Laguerre terms due to change of variable from (-infinity,+infinity) to (0,+infinity)

  Args:
  - u (array): Laguerre terms.

  Returns:
  - Z (array): Combinations of Laguerre terms.
  """
  d = len(u)  # Dimensionality
  aux = np.array(u) * (-1)  # Negate each element of u
  aux = tuple(aux)  # Convert to tuple
  L = [(u[i], aux[i]) for i in range(len(u))]  # Create pairs of (u[i], -u[i])
  Z = list(itertools.product(*L))  # Generate all possible combinations of the pairs
  return np.array(Z)

# Call on min options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [24]:
# Model and payoff parameters
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def call_on_min_constraint_1(R):
    return -1 * R

def call_on_min_constraint_2(R):
    return -1 - np.sum(R)

def VG_constraint(R):
    return 1 + nu * theta @ R - 0.5 * nu * R @ SIGMA @ R

cons = ( {'type': 'ineq', 'fun': VG_constraint},
        {'type': 'ineq', 'fun': call_on_min_constraint_1},
        {'type': 'ineq', 'fun': call_on_min_constraint_2},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-5.83641532 -5.83637836 -5.83641692]


## Pricing using Tensor Product Gauss-Laguerre quadrature in the Fourier space

In [13]:
############### Model and payoff parameters ###############
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature nodes per dimension
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
LagQUAD_estimate =  LagQUAD_fourier_VG_call_on_min_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R)
print("LagQUAD estimate =", round(LagQUAD_estimate,5)  )

LagQUAD estimate = 1.77631


# Basket Put options

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [14]:
# Model and payoff parameters
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def basket_put_constraint(R):
    return R

def VG_constraint(R):
    return 1 + nu * theta @ R - 0.5 * nu * R @ SIGMA @ R

cons = ( {'type': 'ineq', 'fun': VG_constraint},
      {'type': 'ineq', 'fun': basket_put_constraint},)

# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_basket_put, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [2.21940094 2.21946564 2.21941962]


In [16]:
############### Model and payoff parameters ###############
K = 100
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature nodes per dimension
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_basket_put, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
LagQUAD_estimate =  LagQUAD_fourier_VG_basket_put_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R)
print("LagQUAD estimate =", round(LagQUAD_estimate,5)  )

LagQUAD estimate = 6.55341


# 1D multi-strike vectorized implementations (Caching)

In [21]:
def multistrike_LagQUAD_fourier_VG_vanilla_call_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R):
  # vectorized implementation of the pricer for Vanilla calls.
  K = np.array(K).reshape([len(K),1]) # list of strikes
  d = 1
  [u,w] = np.polynomial.laguerre.laggauss(N) #Laguerre quadrature nodes and weights.
  u_1d=[u for i in range(d)] #array of 1-D abcissas for all directions
  w_1d=[w for i in range(d)] #array of 1-D weights for all directions
  u = np.asarray(list(product(*u_1d))) # Full Tensor Isotropic Grid of abcissas
  w = np.asarray(list(product(*w_1d))) #Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0,K)) #element-wise division X_0 is logarithm of stock price at initial time
  phi_list = np.zeros(N, dtype = complex) #List of characteristic function evaluations
  p_list = np.zeros(N, dtype = complex) #List of payoff function evaluations
  w_prod_list = np.zeros(N, dtype = float)
  reciprocal_weight_function = np.zeros(N, dtype = float)
  phi_values = np.array(list(map(lambda u: VG_characteristic_function(u + 1j*R, SIGMA, T, r, theta, nu), u) ) )
  p_values = np.array(list(map(fourier_payoff_call_on_min, u+1j*R)))
  w_prod_values = np.prod(w,axis = 1)
  reciprocal_weight_function = np.exp(np.sum(u,axis = 1 ))
  temp = p_values * phi_values * w_prod_values * reciprocal_weight_function
  mat = np.exp(1j * np.outer(X0,u))
  K_factor = (2*np.pi)**(-d) * np.exp(-r*T) * K * np.exp(-R*X0)
  V = 2 * K_factor.reshape(K.shape[0],)  * np.real( mat @ temp ) # The factor 2 comes from evenness of the integrand
  return V

In [38]:
############### Model and payoff parameters ###############
K_grid = list(np.linspace(80,120,40)) # strike prices grid
r = 0
T = 1
nu = 0.2
dimension = 1
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def rainbow_constraint_1(R):
  return -1*R
def rainbow_constraint_2(R):
  return -1 - np.sum(R)
cons = ( {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
K = np.median(K_grid)
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "trust-constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
LagQUAD_estimates = multistrike_LagQUAD_fourier_VG_vanilla_call_pricer(S0,K_grid,r,T,sigma,rho,theta,nu,SIGMA,N,R)
print("LagQUAD estimates =", LagQUAD_estimates )

Optimal damping parameters: [-7.70210902]
LagQUAD estimates = [22.29791168 21.47390498 20.66417287 19.86886685 19.08812647 18.32211454
 17.57103641 16.83514598 16.11474205 15.41015828 14.72174953 14.04987716
 13.39489487 12.75713633 12.13690516 11.53446752 10.95004725 10.38382312
  9.83592806  9.30644951  8.79543096  8.3028738   7.82873952  7.37295189
  6.9353989   6.51593452  6.11438016  5.73052585  5.36413122  5.01492638
  4.6826127   4.36686367  4.06732582  3.78361984  3.51534188  3.2620651
  3.02334145  2.79870373  2.58766783  2.38973516]


# Multidimensional multistrike vectorized implementation (caching)

In [36]:
def phase_factor(u,X0):
  return np.exp(1j * u @ X0.T )

def multistrike_LagQUAD_fourier_VG_call_on_min_pricer(S0,K,r,T,sigma,rho,theta,nu,SIGMA,N,R):
  d=len(S0) #number of underlying stocks / dimensionality of the problem
  K = np.array(K).reshape([len(K),1]) # list of strikes to be valued simulatenously
  k = K.shape[0] # number of strikes to be valued
  [u,w] = np.polynomial.laguerre.laggauss(N) #returns Laguerre quadrature nodes and weights.
  u_1d=[u for i in range(d)] # Nxd array of 1D abcissas
  w_1d=[w for i in range(d)] # Nxd array of 1D weights
  u = np.asarray(list(product(*u_1d))) # N^d x d: array of N^d d-dimensional points
  w = np.asarray(list(product(*w_1d))) # N^d x d: array of N^d d-dimensional weights
  u_laguerre_scaling = np.repeat(a = u, repeats = 2**d ,axis = 0) # Repeats each element of the array 2**d times before moving to next element
  w = np.repeat(a = w, repeats = 2**d ,axis = 0) # Repeats each element of the array 2**d times before moving to next element
  u = np.vstack(np.array(list(map(laguerre_combinations, u)))) # creates all possible combinations of multiplying by -1.
  X0 = np.log(np.divide(S0,K)) #element-wise division X_0 is logarithm of stock price at initial time - Division by d is intrinsic to basket option for equally weighted average
  phi_values = np.array(list(map(lambda u: VG_characteristic_function(u + 1j*R, SIGMA, T, r, theta, nu), u) ) , dtype = complex) # characteristic function evaluations
  p_values = np.array(list(map(fourier_payoff_call_on_min, u+1j*R)), dtype = complex) # payoff transform evaluations
  w_prod_values = np.prod(w,axis = 1) # contains product of weights for each multi-index
  reciprocal_weight_function = np.exp(np.sum(u_laguerre_scaling,axis = 1 ))  # contains the inverse of the weight function applied to use Gauss-Laguerre.
  temp =  p_values * phi_values * w_prod_values * reciprocal_weight_function
  mat = np.array(list(map(lambda u: phase_factor(u,X0), u)))
  K_factor = (2*np.pi)**(-d) * np.exp(-r*T) * K *  np.exp(-1*np.einsum("ij,ij->i", np.tile(R,(k,1)), X0)).reshape(k,1) # Einstein sumation allows for dot product row-by-row between two matrices. Tiling repeats the vector k times, axis = 1 is to make sure we copy rows.
  V = K_factor  * np.real(np.einsum("ij,ij->i", mat.T, np.tile(temp,(k,1)))).reshape(k,1)
  return V

In [44]:
############### Model and payoff parameters ###############
K_grid = list(np.linspace(80,120,40)) # strike prices grid
r = 0
T = 1
nu = 0.2
dimension = 3
S0 = 100 * np.ones(dimension)
sigma = 0.2*np.ones(dimension)
theta = -0.3*np.ones(dimension)
rho = np.identity(dimension)
SIGMA = covariance_matrix(sigma,rho)
############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def rainbow_constraint_1(R):
  return -1*R
def rainbow_constraint_2(R):
  return -1 - np.sum(R)
cons = ( {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
K = np.median(K_grid)
optimal_R = optimize.minimize(fun = integrand_to_optimize_VG_call_on_min, constraints = cons, x0 = R_init , method = "Nelder-Mead" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

############### Laguerre Quadrature parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
LagQUAD_estimates = multistrike_LagQUAD_fourier_VG_call_on_min_pricer(S0,K_grid,r,T,sigma,rho,theta,nu,SIGMA,N,R)
print("LagQUAD estimates =", LagQUAD_estimates )

Optimal damping parameters: [-5.83641532 -5.83637836 -5.83641692]
LagQUAD estimates = [[9.71032593]
 [9.09980852]
 [8.51215251]
 [7.94719344]
 [7.40477237]
 [6.88474686]
 [6.38699713]
 [5.91142704]
 [5.4579595 ]
 [5.02652562]
 [4.61704851]
 [4.22942391]
 [3.86350067]
 [3.51906395]
 [3.19582335]
 [2.89340685]
 [2.61136035]
 [2.34915173]
 [2.10617803]
 [1.8817745 ]
 [1.67522442]
 [1.48576879]
 [1.31261574]
 [1.15494916]
 [1.01193651]
 [0.88273598]
 [0.76650274]
 [0.66239454]
 [0.56957665]
 [0.48722615]
 [0.41453582]
 [0.35071753]
 [0.29500545]
 [0.24665897]
 [0.20496561]
 [0.16924374]
 [0.13884536]
 [0.11315871]
 [0.09161059]
 [0.07366853]]
